In [3]:
import pandas as pd

df = pd.read_csv("../data/merged.csv")
df.head()


,cst_dim_id,transdate,transdatetime,amount,docno,direction,target,transdate_dt,transdate_beh,monthly_os_changes,...,login_frequency_30d,freq_change_7d_vs_mean,logins_7d_over_30d_ratio,avg_login_interval_30d,std_login_interval_30d,var_login_interval_30d,ewm_login_interval_7d,burstiness_login_interval,fano_factor_login_interval,zscore_avg_login_interval_7d
0,2.937833e+09,'2025-01-05 00:00:00.000','2025-01-05 16:32:02.000',31000.0,5343,8406e407421ec28bd5f445793ef64fd1,0,2025-01-05,'2025-01-05 00:00:00.000',1.0,...,1.533333,0.211180,0.282609,49814.117647,106759.606690,1.139761e+10,18227.846189,0.363698,228802.880771,-0.213134
1,2.096229e+09,'2025-03-04 00:00:00.000','2025-03-04 17:41:57.000',4000.0,8442,b3a3d4a6006293195d998957d4f01e42,0,2025-03-04,'2025-03-04 00:00:00.000',1.0,...,1.766667,-0.110512,0.207547,50667.857143,78912.726908,6.227218e+09,6872.638130,0.217971,122902.739907,0.026791
2,2.937760e+09,'2025-06-20 00:00:00.000','2025-06-20 10:08:07.000',3000.0,9540,22b84292f0ebce65ad0808342615a03b,0,2025-06-20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.933493e+09,'2025-07-06 00:00:00.000','2025-07-06 14:52:13.000',500.0,11685,d677d4e1a0f625e1ad746ea950c9dca9,0,2025-07-06,'2025-07-06 00:00:00.000',0.0,...,0.000000,NaN,NaN,1350.000000,1824.335495,3.328200e+06,NaN,0.149428,2465.333333,NaN
4,4.560006e+08,'2024-12-18 00:00:00.000','2024-12-18 14:12:25.000',20000.0,7128,87b698d1edae13c21ce86678de3b8546,0,2024-12-18,'2024-12-18 00:00:00.000',1.0,...,0.433333,1.307692,0.538462,213360.000000,425545.970020,NaN,127828.033176,0.332108,848750.340270,-0.408581


In [4]:
df["target"].value_counts(normalize=True)


target
0    0.987417
1    0.012583
Name: proportion, dtype: float64

In [5]:
from datetime import datetime

def strip_quotes(s: str) -> str:
    s = str(s).strip()
    if s.startswith("'") and s.endswith("'"):
        s = s[1:-1]
    return s

df["transdatetime_dt"] = pd.to_datetime(df["transdatetime"].apply(strip_quotes))

df["hour"] = df["transdatetime_dt"].dt.hour
df["dayofweek"] = df["transdatetime_dt"].dt.dayofweek


In [6]:
target_col = "target"

cols_to_drop = [
    target_col,
    "transdate",        # строковая дата
    "transdatetime",    # строковая дата-время
    "transdate_dt",     # уже использовали для merge
    "transdatetime_dt", # служебное поле
]

X = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
y = df[target_col]

# Категориальные признаки (строки)
cat_cols = [c for c in X.columns if X[c].dtype == "object"]

# Заполняем пропуски в категориальных
for c in cat_cols:
    X[c] = X[c].fillna("NA").astype(str)

cat_indices = [X.columns.get_loc(c) for c in cat_cols]

X.head()


,cst_dim_id,amount,docno,direction,transdate_beh,monthly_os_changes,monthly_phone_model_changes,last_phone_model_categorical,last_os_categorical,logins_last_7_days,...,logins_7d_over_30d_ratio,avg_login_interval_30d,std_login_interval_30d,var_login_interval_30d,ewm_login_interval_7d,burstiness_login_interval,fano_factor_login_interval,zscore_avg_login_interval_7d,hour,dayofweek
0,2.937833e+09,31000.0,5343,8406e407421ec28bd5f445793ef64fd1,'2025-01-05 00:00:00.000',1.0,1.0,"iPhone16,1",iOS/17.5.1,13.0,...,0.282609,49814.117647,106759.606690,1.139761e+10,18227.846189,0.363698,228802.880771,-0.213134,16,6
1,2.096229e+09,4000.0,8442,b3a3d4a6006293195d998957d4f01e42,'2025-03-04 00:00:00.000',1.0,1.0,Samsung SM-S918B,Android/14,11.0,...,0.207547,50667.857143,78912.726908,6.227218e+09,6872.638130,0.217971,122902.739907,0.026791,17,1
2,2.937760e+09,3000.0,9540,22b84292f0ebce65ad0808342615a03b,NA,NaN,NaN,NA,NA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,4
3,2.933493e+09,500.0,11685,d677d4e1a0f625e1ad746ea950c9dca9,'2025-07-06 00:00:00.000',0.0,0.0,Vivo_v2339,Android/15,0.0,...,NaN,1350.000000,1824.335495,3.328200e+06,NaN,0.149428,2465.333333,NaN,14,6
4,4.560006e+08,20000.0,7128,87b698d1edae13c21ce86678de3b8546,'2024-12-18 00:00:00.000',1.0,1.0,Samsung SM-A536E,Android/14,7.0,...,0.538462,213360.000000,425545.970020,NaN,127828.033176,0.332108,848750.340270,-0.408581,14,2


In [7]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42,
)

model = CatBoostClassifier(
    depth=6,
    learning_rate=0.1,
    loss_function="Logloss",
    eval_metric="AUC",
    class_weights=[1, 10],  # усиливаем вес мошенничества
    verbose=100,
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    cat_features=cat_indices,
    use_best_model=True,
)


0:	test: 0.7344696	best: 0.7344696 (0)	total: 192ms	remaining: 3m 11s
100:	test: 0.9887642	best: 0.9889907 (89)	total: 5s	remaining: 44.5s
200:	test: 0.9887642	best: 0.9891864 (118)	total: 9.78s	remaining: 38.9s
300:	test: 0.9890216	best: 0.9891864 (118)	total: 14.5s	remaining: 33.8s
400:	test: 0.9893975	best: 0.9894027 (394)	total: 19.4s	remaining: 28.9s
500:	test: 0.9891349	best: 0.9894490 (402)	total: 24.2s	remaining: 24.1s
600:	test: 0.9887796	best: 0.9894490 (402)	total: 28.9s	remaining: 19.2s
700:	test: 0.9887951	best: 0.9894490 (402)	total: 33.6s	remaining: 14.3s
800:	test: 0.9888620	best: 0.9894490 (402)	total: 38.3s	remaining: 9.52s
900:	test: 0.9888568	best: 0.9894490 (402)	total: 43.1s	remaining: 4.73s
999:	test: 0.9888311	best: 0.9894490 (402)	total: 47.8s	remaining: 0us

bestTest = 0.9894490216
bestIteration = 402

Shrink model to first 403 iterations.


In [9]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# 1) вероятности класса "fraud"
proba_val = model.predict_proba(X_val)[:, 1]

print("ROC-AUC:", roc_auc_score(y_val, proba_val))

# 2) пока возьмём порог 0.5 (потом подберём)
y_pred = (proba_val > 0.5).astype(int)
print(classification_report(y_val, y_pred))

print(confusion_matrix(y_val, y_pred))


ROC-AUC: 0.9894490216271884
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3884
           1       0.82      0.28      0.42        50

    accuracy                           0.99      3934
   macro avg       0.91      0.64      0.71      3934
weighted avg       0.99      0.99      0.99      3934

[[3881    3]
 [  36   14]]


In [10]:
from pathlib import Path

Path("../src").mkdir(exist_ok=True)

model.save_model("../src/fraud_model.cbm")


In [11]:
import json

feature_info = {
    "feature_names": list(X.columns),
    "cat_features": [int(i) for i in cat_indices],
}

with open("../src/feature_info.json", "w", encoding="utf-8") as f:
    json.dump(feature_info, f, ensure_ascii=False, indent=2)


In [12]:
sample = X_val.iloc[[0]]
print("True label:", y_val.iloc[0])
print("Proba:", model.predict_proba(sample)[:, 1][0])
print("Pred:", model.predict(sample)[0])


True label: 0
Proba: 1.2648193321417213e-06
Pred: 0


In [13]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# вероятности фрода
proba_val = model.predict_proba(X_val)[:, 1]

print("ROC-AUC:", roc_auc_score(y_val, proba_val))

# пока порог 0.5
y_pred = (proba_val > 0.5).astype(int)

print("\nClassification report:")
print(classification_report(y_val, y_pred))

print("\nConfusion matrix:")
print(confusion_matrix(y_val, y_pred))


ROC-AUC: 0.9894490216271884

Classification report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3884
           1       0.82      0.28      0.42        50

    accuracy                           0.99      3934
   macro avg       0.91      0.64      0.71      3934
weighted avg       0.99      0.99      0.99      3934


Confusion matrix:
[[3881    3]
 [  36   14]]


In [14]:
import pandas as pd

feature_importances = model.get_feature_importance()
fi_df = pd.DataFrame({
    "feature": X.columns,
    "importance": feature_importances
}).sort_values("importance", ascending=False)

fi_df.head(20)


,feature,importance
3,direction,29.072694
1,amount,13.077991
4,transdate_beh,6.953162
7,last_phone_model_categorical,5.443450
8,last_os_categorical,5.261818
22,hour,5.202354
2,docno,4.078593
23,dayofweek,3.507280
0,cst_dim_id,3.351860
20,fano_factor_login_interval,3.000436


In [15]:
fi_df.to_csv("../data/feature_importances.csv", index=False)


In [16]:
from pathlib import Path
import json

Path("../src").mkdir(exist_ok=True)

# сама модель
model.save_model("../src/fraud_model.cbm")

# инфа о фичах
feature_info = {
    "feature_names": list(X.columns),
    "cat_indices": [int(i) for i in cat_indices],
}
with open("../src/feature_info.json", "w", encoding="utf-8") as f:
    json.dump(feature_info, f, ensure_ascii=False, indent=2)
